<a href="https://colab.research.google.com/github/virajvaidya/Australia-Energy-Demand-Forecasting/blob/main/TASPIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.3.5

In [ ]:
!pip install plotly==5.4.0
!pip install chart_studio

In [ ]:
!pip install statsmodels==0.13.1

In [ ]:
!pip install pmdarima

In [6]:
import seaborn              as sns
import pandas               as pd
import numpy                as np
import pmdarima             as pm
import plotly.graph_objs    as go

from pmdarima        import pipeline
from pmdarima        import model_selection
from pmdarima        import preprocessing as ppc
from pmdarima        import arima
from matplotlib      import pyplot as plt
from plotly.offline  import iplot
from plotly.subplots import make_subplots

In [7]:
sns.set_style('whitegrid')
pd.plotting.register_matplotlib_converters()
# Default figure size
sns.mpl.rc('figure',figsize=(16, 6))

In [8]:
from google.colab import files
import io
uploaded = files.upload()

Saving TAS.csv to TAS.csv


In [9]:
df = pd.read_csv('TAS.csv')

In [11]:
df= df[['DATE','TOTALDEMANDTAS','RRPTAS']]

In [ ]:
df['DATE'] = df['DATE'].apply(lambda x: pd.Timestamp(x).strftime('%d/%m/%Y %H%M'))

In [ ]:
df['DATE'] = df['DATE'].astype('datetime64[ns]')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70942 entries, 0 to 70941
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            70942 non-null  datetime64[ns]
 1   TOTALDEMANDTAS  70942 non-null  float64       
 2   RRPTAS          70942 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ MB


In [15]:
data = df['TOTALDEMANDTAS']
train, test = model_selection.train_test_split(data, train_size=40000)

In [16]:
pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=12, k=4)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                              information_criterion='bic',
                              seasonal=False,  # because we use Fourier
                              suppress_warnings=True))
])

In [17]:
pipe.fit(train)
print("Model fit:")
print(pipe)

Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=361481.643, Time=85.17 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=385130.148, Time=3.37 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=367294.973, Time=3.63 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=373614.163, Time=24.54 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=385119.555, Time=29.40 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : BIC=366361.532, Time=21.94 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : BIC=366468.644, Time=27.44 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : BIC=361460.903, Time=112.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : BIC=366461.127, Time=86.98 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : BIC=361698.085, Time=162.96 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : BIC=361493.382, Time=127.80 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : BIC=361462.105, Time=102.89 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : BIC=361637.215, Time=129.67 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : BIC=361482.102, Tim

In [18]:
preds, conf_int = pipe.predict(n_periods=100, return_conf_int=True)
print("\nForecasts:")
print(preds)


Forecasts:
[1324.19017453 1299.57736341 1272.80639223 1247.3472949  1225.5075643
 1209.58382776 1198.58785001 1189.61650353 1181.83012147 1175.32111374
 1170.21941694 1166.4962578  1162.82758939 1159.45250993 1159.22485612
 1163.54034966 1172.67160448 1187.04952072 1204.07505055 1219.5493144
 1231.68741554 1239.99883494 1244.38309302 1244.90521775 1240.61611766
 1232.35729265 1223.75230661 1217.07150469 1213.50603204 1214.3929061
 1217.97772628 1220.80488747 1221.70179581 1220.64041837 1217.82465134
 1213.46727368 1206.62036326 1197.99704938 1190.98525555 1187.5380686
 1188.47496705 1194.73395693 1204.15971209 1212.91635977 1219.49047978
 1223.56704709 1225.12651456 1224.22680416 1219.83435104 1212.64255353
 1206.0789219  1202.18631853 1201.91332733 1206.35460247 1213.52706302
 1219.77126015 1223.74405437 1225.28578293 1224.51042484 1221.58269731
 1215.54626745 1207.14134859 1199.81278737 1195.59377753 1195.39992139
 1200.27413615 1208.16792056 1215.34796752 1220.39394174 1223.0703382